# Laboratorio 6
## Data Science

Autores:

- Nelson García 
- Christian Echeverría

## Análisis de Sentimientos de críticas de películas

Junto con Keras, viene un ejemplo imdb_lstm.py. Este ejercicio esta prácticamente basado en él.

Es un gran ejemplo del uso de las RNNs.  El conjunto de datos que se utilizará consta de críticas de películas generadas por usuarios, y una classificación indicando si le gustó, o no, basado en su rating asociado. 

Hay más información de este conjunto de datos en:

https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

Como la comprensión del lenguaje escrito requiere "llevar cuenta" de todas las palabras en una oración, necesitamos una RNN para mantener una "memoria" de las palabras que pasaron antes, conforme va "leyendo" oraciones a lo largo del tiempo. 

En particular, se usarán unidades LSTM (Long Short-Term Memory) porque no es deseable "olvidar" palabras demasiado rápido...las palabras al inicio de una oración pueden afectar el significado de la misma grandemente.

Empezamos por la importación de lo que se requiere:

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"   # habilita growth desde el arranque
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"           

In [2]:
import tensorflow as tf

print("TF:", tf.__version__)
print("Build info:", getattr(tf.sysconfig, "get_build_info", lambda: {})())

gpus = tf.config.list_physical_devices('GPU')
print("GPUs detectadas:", gpus)

# Activa growth explícitamente (ya debería estar por env var, pero así confirmas)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

TF: 2.20.0
Build info: OrderedDict({'cpu_compiler': 'clang 18', 'cuda_compute_capabilities': ['sm_60', 'sm_70', 'sm_80', 'sm_89', 'compute_90'], 'cuda_version': '12.5.1', 'cudnn_version': '9', 'is_cuda_build': True, 'is_rocm_build': False, 'is_tensorrt_build': False})
GPUs detectadas: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Ahora importar los datos para entrenamiento y prueba.  Para que sea más manejable, se especifica que se quieren solamente las 50,000 palabras más populares en el conjunto de datos. Por algún motivo, este conjunto tiene una relación de 50% entreno y 50% prueba. 

In [4]:
print('Cargando los datos...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=50000)

Cargando los datos...


In [5]:
len(X_train)

25000

In [6]:
len(X_test)

25000

A ver cómo son los datos, el primer elemento de entrenamiento debe ser una crítica de una película:

In [10]:
X_train[0]

array([   15,   256,     4, 31050,     7,  3766,     5,   723,    36,
          71,    43,   530,   476,    26,   400,   317,    46,     7,
           4, 12118,  1029,    13,   104,    88,     4,   381,    15,
         297,    98,    32,  2071,    56,    26,   141,     6,   194,
        7486,    18,     4,   226,    22,    21,   134,   476,    26,
         480,     5,   144,    30,  5535,    18,    51,    36,    28,
         224,    92,    25,   104,     4,   226,    65,    16,    38,
        1334,    88,    12,    16,   283,     5,    16,  4472,   113,
         103,    32,    15,    16,  5345,    19,   178,    32],
      dtype=int32)

Esto no parece una crítica de una película!!!!  Resulta que la gente que preparó los datos ya hizo algo de preparación previa de los datos.  Estos números coinciden con el índice correspondiente a cada palabra de la crítica.  En realidad las palabras en sí, no son de interés...el modelo requiere números no palabras. 

Lo triste es que no será posible leer las críticas...siquiera para tener una idea de si funciona el análiisis, o no.

Y, ¿cómo son las etiquetas (metas)?

In [11]:
y_train[0]

np.int64(1)

Son simplemente 0 ó 1, que indica sí al que escribió la crítica le gustó, o no, la película.

Para resumir, para el entrenamiento se tiene un conjunto de críticas de películas que han sido convertidas a vectores de palabras representadas por enteros, y una clasificación de sentimiento binaria.

Las RNNs pueden "explotar" muy rápidamente (se habló de esto en clase).  Para que no se sobrecarguen las PCs que se podrían usar, se limitarán las críticas a las primeras 80 palabras:

In [12]:
X_train = sequence.pad_sequences(X_train, maxlen = 80)
X_test = sequence.pad_sequences(X_test, maxlen = 80)

## Configuración del modelo

Considerando lo complejo que es una RNN LSTM (debajo del "capó"), es increíble lo simple que resulta esta parte utilizando Keras.

Se empieza con una capa de incrustación "embedded".  La función de esta es convertir los datos de entrada a vectores densos de un tamaño fijo que son más adecuados para una red neuronal.  Esto es típico cuando se manejan indices de datos basados en texto.  El 20,000 representa el tamaño del vocabulario (límite impuesto para este ejercicio, pero puede variarse dependiendo de la capacidad de cómputo que se tiene) y 128 es la dimensión de 128 unidades de salida.

Luego se coloca una capa LSTM y ya!  Se especifica 128 para igualar la salida de la capa de incrustación, y se utiliza la opción de "botar" unidades para evitar el sobre ajuste, que es una característica muy común de las RNN.

Finalmente se debe reducir todo a una unidad de salida con una función de activación sigmoidal para seleccionar la clasificación binaria de sentimiento de 0 ó 1.

In [13]:
modelo = Sequential()
modelo.add(Embedding(50000, 128))
modelo.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
modelo.add(Dense(1, activation='sigmoid'))

Ya que este es un problema de clasificación binaria, la mejor función de pérdida es la "binary_crossentropy".  También se utiliza el optimizador "Adam" que es uno de los mejores.  Siempre es de recordar que si es necesario afinar más el modelo, se pueden probar otras.

In [14]:
modelo.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Entrenamiento

Las RNNs, al igual que las CNNs son bastante pesadas en el uso de recursos.  Para poder utilizar una PC normal, el mantener el tamaño de las tandas pequeño, es clave.  En un mundo más profesional, se estaría sacando ventaja de GPUs instaladas en un cluster de computadoras para tener un mejor rendimiento.


Ahora sí, a iniciar el entrenamiento.  Aún con una GPU se tardará bastante tiempo!

In [17]:
# Agregar detención temprana para monitorear el entrenamiento
detencion_temprana = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 3,
    verbose = 1
)

In [18]:
historia = modelo.fit(
    X_train, 
    y_train,
    batch_size = 64,  # Puede ajustarse de acuerdo a la memoria de GPU disponible 
    epochs = 15,
    verbose = 1,     
    validation_data = (X_test, y_test),
    callbacks = [detencion_temprana]
)

Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 126s 304ms/step - accuracy: 0.7755 - loss: 0.4658 - val_accuracy: 0.8365 - val_loss: 0.3722
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 95s 244ms/step - accuracy: 0.8897 - loss: 0.2765 - val_accuracy: 0.8270 - val_loss: 0.4077
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 107s 273ms/step - accuracy: 0.9332 - loss: 0.1796 - val_accuracy: 0.8280 - val_loss: 0.4117
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 102s 260ms/step - accuracy: 0.9577 - loss: 0.1191 - val_accuracy: 0.8237 - val_loss: 0.5160
Epoch 4: early stopping


In [19]:
# Después del entrenamiento se puede revisar el historial
print(historia.history.keys())

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


**NOTA:** Si se entrenó el modelo en esta sesión y se desea guardar para otra aplicación o uso futuro, Ejecute la instrucción de la siguiente celda.  Si no lo entrenó en esta sesión, omita la ejecución. 



In [20]:
modelo.save("Analisis_Sentimiento.keras")

**NOTA:** Si no se entrenó el modelo en esta sesión, pero lo tiene guardado, puede ejecutar la instrucciones de la siguiente celda.

In [21]:
from tensorflow.keras.models import load_model
modelo = load_model("Analisis_Sentimiento.keras")

OK, ahora a evaluar la exactitud del modelo:

In [22]:
perdida, exactitud = modelo.evaluate(X_test, y_test,
                            batch_size = 64,
                            verbose = 2)
print('Pérdida de la Prueba:', perdida)
print('Exactitud de la Prueba (Test accuracy):', exactitud)

391/391 - 12s - 30ms/step - accuracy: 0.8237 - loss: 0.5160
Pérdida de la Prueba: 0.5159767866134644
Exactitud de la Prueba (Test accuracy): 0.8236799836158752


Cerca del 80%, no está mal considerando que se limitó el ejercicio a las primeras 80 palabras de cada crítica.

Hay que pensar en lo que se hizo en este ejercicio!  Una red neuronal que puede leer críticas y deducir si al autor le gustó la película o no, basado en el texto.  Y el modelo toma en cuenta el contexto y la posición de cada palabra.

Lo mejor de todo es que armar el modelo solamente requirió de algunas líneas de código!  Es increíble lo que se puede hacer con Keras!!!

### Ver algo de las críticas

Del enlace provisto arriba, se obtuvo el siguiente código que permite ver el texto del primero de los comentarios.  No sería dificil verlos todos si se quisiera.

In [23]:
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3
# Retrieve the training sequences.
(X_train, _), _ = tf.keras.datasets.imdb.load_data(
    start_char=start_char, oov_char=oov_char, index_from=index_from
)
# Retrieve the word index file mapping words to indices
word_index = tf.keras.datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in X_train[0])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [24]:
decoded_sequence

"[START] this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and sh